# Step 1: Read & Process Transcript Files
We first read the transcript text files, grouping together the files in the "Free" group as one set, and the "D-C" group as another set.

To avoid processing errors, we first remove all non-UTF characters in the text (such as distinct opening and closing quotes)
and replace them with corresponding UTF characters. These transcripts also come with the transcriber's observations, such as [laughter], or [silent]. We remove all such observations so that we are only looking at team members' speech.

Finally we remove all stop words

In [1]:
import re
# Locations of input files. These files have been anonymized, and the files themselves will not be available
# in this folder, to preserve the privacy of the teams.
textPathFree = "../../../private/brainstorming_viz_data/free_group/"
textPathDC = "../../../private/brainstorming_viz_data/d-c_group/"

# Teamwise transcript (of divergent processes) of the "Free Brainstorming" group
fileNamesFreeDivergent = ['teamA_part1_divergent.txt', 'teamB_part1_divergent.txt', 'teamC_part1_divergent.txt',
                          'teamD_part1_divergent.txt', 'teamE_part1_divergent.txt', 'teamF_part1_divergent.txt',
                          'teamG_part1_divergent.txt', 'teamH_part1_divergent.txt', 'teamI_part1_divergent.txt']

# Teamwise transcript (of divergent processes) of the "Divergent-Convergent (D-C) Brainstorming" group
fileNamesDCDivergent = ['teamJ_part1_divergent.txt', 'teamJ_part3_divergent.txt', 'teamK_part1_divergent.txt',
                        'teamK_part3_divergent.txt', 'teamL_part1_divergent.txt', 'teamL_part3_divergent.txt', 
                        'teamM_part1_divergent.txt', 'teamM_part2_3_convergent_divergent.txt', 'teamN_part1_divergent.txt',
                        'teamN_part3_divergent.txt', 'teamO_part1_divergent.txt', 'teamO_part2_3_conv_div.txt',
                        'teamP_part1_divergent.txt', 'teamP_part3_divergent.txt', 'teamQ_part1_divergent.txt',
                        'teamQ_part3_4_div_conv.txt', 'teamR_part1_divergent.txt', 'teamR_part3_divergent.txt'] 

# Teamwise transcript (of divergent processes) of the "Free Brainstorming" group
fileNamesFreeConvergent = ['teamA_part2_convergent.txt', 'teamB_part2_convergent.txt', 'teamC_part2_convergent.txt',
                           'teamD_part2_convergent.txt', 'teamE_part2_convergent.txt', 'teamF_part2_convergent.txt',
                           'teamG_part2_convergent.txt', 'teamH_part2_convergent.txt', 'teamI_part2_convergent.txt']

# Teamwise transcript (of divergent processes) of the "Divergent-Convergent (D-C) Brainstorming" group
fileNamesDCConvergent = ['teamJ_part2_convergent.txt', 'teamJ_part4_convergent.txt', 'teamK_part2_convergent.txt',
                         'teamK_part4_convergent.txt', 'teamL_part2_convergent.txt', 'teamL_part4_convergent.txt',
                         'teamM_part4_convergent.txt', 'teamN_part2_convergent.txt', 'teamN_part4_convergent.txt',
                         'teamO_part4_convergent.txt', 'teamP_part2_convergent.txt', 'teamP_part4_convergent.txt',
                         'teamQ_part2_convergent.txt', 'teamR_part2_convergent.txt', 'teamR_part4_convergent.txt']  

def openFiles(fileNamesList, filePath):
    # Given a list of file names and a path for the files, returns a
    # list of file objects
    fObjList = []
    for fileName in fileNamesList:
        fObj = open(filePath + fileName, 'r', encoding='utf8')
        fObjList.append(fObj)
    return fObjList

filesListFree = openFiles(fileNamesFreeDivergent + fileNamesFreeConvergent, textPathFree)
filesListDC = openFiles(fileNamesDCDivergent + fileNamesDCConvergent, textPathDC)

# Some of these files have non-UTF characters that may need to be removed.
def removeSpecials(filesList):
    # Remove some incompatible unicode characters such as distinct
    # opening and closing quotes, ellipsis etc.
    # Also finally converts all text to lowercase
    outString = ''
    for inFile in filesList:
        # print("reading file",inFile.name.split("/")[-1], "...", end="")
        temp = inFile.read()
        subsFileQuotes = re.sub(u"(\u2018|\u2019)", "'", temp)
        subsFileDQuotes = re.sub(u'(\u201c|\u201d)', '"', subsFileQuotes)
        subsFileEllipsis = re.sub(u"\u2026", "...", subsFileDQuotes)
        outString += subsFileEllipsis
        # print("done.")
    return outString.lower()

group1_raw = removeSpecials(filesListFree)
group2_raw = removeSpecials(filesListDC)

# These files have the transcriber's observations like [laughter], [silent], [cross-talk] etc.
# These may throw off the text analysis, so we remove them. These are typically within parantheses,
# so we can remove everything between these parantheses.

def removeObservations(test_str) :
    # removes all text within parantheses
    # these are usually observations by the transcriber
    # and thus should not be considered in content analysis
    # code from http://stackoverflow.com/questions/14596884/remove-text-between-and-in-python
    ret = ''
    skip1c = 0
    skip2c = 0
    squareparen = 0
    for i in test_str:
        if i == '[':
            skip1c += 1
            squareparen += 1
        elif i == '(':
            skip2c += 1
        elif i == ']' and skip1c > 0:
            skip1c -= 1
        elif i == ')'and skip2c > 0:
            skip2c -= 1
        elif skip1c == 0 and skip2c == 0:
            ret += i
    return ret

group1 = removeObservations(group1_raw)
group2 = removeObservations(group2_raw)

group1_lines = group1.split('\n')
group2_lines = group2.split('\n')

print("Number of lines in the Free Group:", len(group1_lines))
print("Number of lines in the D-C Group:", len(group2_lines))
print("Each line will be considered a separate document for computing the Document-Term Matrix")

Number of lines in the Free Group: 7641
Number of lines in the D-C Group: 3872
Each line will be considered a separate document for computing the Document-Term Matrix


# Step 2: Computing the Latent Dirichlet Allocation
Detailed explanation to follow...


In [2]:
from __future__ import print_function
import pyLDAvis
import pyLDAvis.sklearn
pyLDAvis.enable_notebook()
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from nltk.corpus import stopwords

C:\Anaconda3\lib\site-packages\funcy\decorators.py:56: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() instead
  spec = inspect.getargspec(func)
C:\Anaconda3\lib\site-packages\funcy\decorators.py:56: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() instead
  spec = inspect.getargspec(func)
C:\Anaconda3\lib\site-packages\funcy\decorators.py:56: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() instead
  spec = inspect.getargspec(func)
C:\Anaconda3\lib\site-packages\funcy\decorators.py:56: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() instead
  spec = inspect.getargspec(func)
C:\Anaconda3\lib\site-packages\funcy\decorators.py:56: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() instead
  spec = inspect.getargspec(func)
C:\Anaconda3\lib\site-packages\funcy\decorators.py:56: DeprecationWarning: inspect.getargspec() is deprecated, use 

## Step 2.1 Computing the Document-Term Matrix

The document-term matrix looks like this:

|      | T1     | T2     | T3     | T4     | T5     |... 
|:----:|:------:|:------:|:------:|:------:|:------:|:---:
| D1   | N_d1t1 | N_d1t2 | N_d1t2 | N_d1t3 | N_d1t4 |...
| D2   | N_d2t1 | N_d2t2 | N_d2t2 | N_d2t3 | N_d2t4 |...
| D3   | N_d3t1 | N_d3t2 | N_d3t2 | N_d3t3 | N_d3t4 |...
| D4   | N_d4t1 | N_d4t2 | N_d4t2 | N_d4t3 | N_d4t4 |...
| D5   | N_d5t1 | N_d5t2 | N_d5t2 | N_d5t3 | N_d5t4 |...
| D6   | N_d6t1 | N_d6t2 | N_d6t2 | N_d6t3 | N_d6t4 |...
| ..   | ...... | ...... | ...... | ...... | ...... |...

Where N_ditj is called the **term frequency**, and is equal to the the number of occurrences of term Tj in document Di.

For our purpose, each term is a word longer than two letters, and each document is a line in the transcript. We need to focus on words that matter, so this means removing any stop words, as well as words from the transcript that we deem irrelevant. These include filler words such as "yeah", "like", etc. and non-words such as "hmm". Since this is a transcript, we find a number of time-stamped entries embedded in the text, so these need to go as well. A complete list of words to remove from the list of terms is below (excluding stop words).

Each document is represented as an M-dimensional vector where M is the number of terms computed for the corpus. Here, the vector corresponds to the row of the document-term matrix.

In [17]:
words_to_remove = ['yea', 'like', 'up', 'down', "!", "$", "'", "''", "'cause", 
                   "'d", "'em", "'ll", "'m", "'re", "'s", "'ve", ",", "yeah",
                   "-", ".", "1", "10", "12", "12.", "15", "20", "3-d", 
                   "30", "300", "352.", "36", "375", "3d", "452", "475", 
                   "475.", "90", ":", ";", "?", "``", "one", "would", "right",
                   "okay", 'could', 'know', 'make', 'something', 'yes', 'good',
                   "think", "oh", "maybe", 'kinda', 'actually', 'thing', 'things',
                   'gonna', 'wanna', 'really', 'hmm',
                   # The below set was after topic modeling
                   'cuz', 'huh', 'didn', 'wouldn', 'doesn', 'whatever', 'guys'
                  ]

complete_stop_words = stopwords.words('english') + words_to_remove

tf_vectorizer_grp1 = CountVectorizer(strip_accents = 'unicode',
                                     analyzer = 'word',
                                     stop_words = complete_stop_words,
                                     lowercase = True,
                                     token_pattern = r'\b[a-zA-Z]{3,}\b',
                                     max_df = 0.5, 
                                     min_df = 10)

tf_vectorizer_grp2 = CountVectorizer(strip_accents = 'unicode',
                                     analyzer = 'word',
                                     stop_words = complete_stop_words,
                                     lowercase = True,
                                     token_pattern = r'\b[a-zA-Z]{3,}\b',
                                     max_df = 0.5, 
                                     min_df = 10)
dtm_tf_grp1 = tf_vectorizer_grp1.fit_transform(group1_lines)
print("Document-Term Matrix size for Free Group:", dtm_tf_grp1.shape)
dtm_tf_grp2 = tf_vectorizer_grp2.fit_transform(group2_lines)
print("Document-Term Matrix size for D-C Group:", dtm_tf_grp2.shape)

Document-Term Matrix size for Free Group: (7641, 475)
Document-Term Matrix size for D-C Group: (3872, 285)


## 2.2 Compute the Term Frequency - Inverse Document Frequency
In order to use a scheme for weighting the document-term matrix, we use the popular **Inverse Document Frequency (IDF)** weighting measure.
The IDF is calculated as:

$IDF = log(N / n_j)$

Where $N$ is the total number of documents, and $n_j$ is the number of documents containing term $j$.
This system gives higher weightage to terms that occur in relatively fewer documents (and thus are more characteristic of the *topic* of that document)

The weighted matrix is called Term Frequency-Inverse Document Frequency, or **Tf-idf**, where each cell (term frequency) is multiplied by the weight (inverse document frequency)

In [18]:
tfidf_vectorizer_grp1 = TfidfVectorizer(**tf_vectorizer_grp1.get_params())
dtm_tfidf_grp1 = tfidf_vectorizer_grp1.fit_transform(group1_lines)

tfidf_vectorizer_grp2 = TfidfVectorizer(**tf_vectorizer_grp2.get_params())
dtm_tfidf_grp2 = tfidf_vectorizer_grp2.fit_transform(group2_lines)

## 2.3 Compute Latent Dirichlet Allocation (LDA)
The following explanation is from [Edwin Chen's website](http://blog.echen.me/2011/08/22/introduction-to-latent-dirichlet-allocation/)

LDA represents documents as mixtures of topics that spit out words with certain probabilities. It assumes that documents are produced in the following fashion: when writing each document, you:

* Decide on the number of words N the document will have (say, according to a Poisson distribution).
* Choose a topic mixture for the document (according to a Dirichlet distribution over a fixed set of K topics). For example, assuming that we have the two food and cute animal topics above, you might choose the document to consist of 1/3 food and 2/3 cute animals.
* Generate each word w_i in the document by:
    * First picking a topic (according to the multinomial distribution that you sampled above; for example, you might pick the food topic with 1/3 probability and the cute animals topic with 2/3 probability).
    * Using the topic to generate the word itself (according to the topic’s multinomial distribution). For example, if we selected the food topic, we might generate the word “broccoli” with 30% probability, “bananas” with 15% probability, and so on.
* Assuming this generative model for a collection of documents, LDA then tries to backtrack from the documents to find a set of topics that are likely to have generated the collection.

### Example

Let’s make an example. According to the above process, when generating some particular document D, you might

* Pick 5 to be the number of words in D.
* Decide that D will be 1/2 about food and 1/2 about cute animals. (Number of topics = 2)
* Pick the first word to come from the food topic, which then gives you the word “broccoli”.
* Pick the second word to come from the cute animals topic, which gives you “panda”.
* Pick the third word to come from the cute animals topic, giving you “adorable”.
* Pick the fourth word to come from the food topic, giving you “cherries”.
* Pick the fifth word to come from the food topic, giving you “eating”.
So the document generated under the LDA model will be “broccoli panda adorable cherries eating” (note that LDA is a bag-of-words model).


In [19]:
# for TF DTM
lda_tf_grp1 = LatentDirichletAllocation(n_topics=4, random_state=0)
lda_tf_grp1.fit(dtm_tf_grp1)
lda_tf_grp2 = LatentDirichletAllocation(n_topics=4, random_state=0)
lda_tf_grp2.fit(dtm_tf_grp2)
# for TFIDF DTM
lda_tfidf_grp1 = LatentDirichletAllocation(n_topics=4, random_state=0)
lda_tfidf_grp1.fit(dtm_tfidf_grp1)
lda_tfidf_grp2 = LatentDirichletAllocation(n_topics=4, random_state=0)
lda_tfidf_grp2.fit(dtm_tfidf_grp2)

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
             evaluate_every=-1, learning_decay=0.7,
             learning_method='online', learning_offset=10.0,
             max_doc_update_iter=100, max_iter=10, mean_change_tol=0.001,
             n_jobs=1, n_topics=4, perp_tol=0.1, random_state=0,
             topic_word_prior=None, total_samples=1000000.0, verbose=0)

## 2.4 Visualize Free Brainstorming Topics Using Term Frequencies (unweighted)
Using PyLDAvis, visualize topics and salient terms. To focus on terms that are more salient to each topic, set $\lambda$ to 0.4.

In [20]:
# pyLDAvis.sklearn.prepare(lda_tf_grp1, dtm_tf_grp1, tf_vectorizer_grp1)

In [21]:
# pyLDAvis.sklearn.prepare(lda_tf_grp2, dtm_tf_grp2, tf_vectorizer_grp2)

## 2.4 Visualize Free Brainstorming Topics Using TF-IDF (weighted)
Using PyLDAvis, visualize topics and salient terms. To focus on terms that are more salient to each topic, set $\lambda$ to 0.4.

In [22]:
pyLDAvis.sklearn.prepare(lda_tfidf_grp1, dtm_tfidf_grp1, tfidf_vectorizer_grp1)

PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
1      26.096144        1       1  0.347484 -0.005912
3      25.254211        1       2 -0.225422 -0.030744
0      24.475846        1       3 -0.056472 -0.271071
2      24.173798        1       4 -0.065590  0.307727, topic_info=     Category        Freq        Term       Total  loglift  logprob
term                                                               
241   Default  108.000000        mean  108.000000  30.0000  30.0000
194   Default  107.000000        idea  107.000000  29.0000  29.0000
457   Default   99.000000        well   99.000000  28.0000  28.0000
315   Default   90.000000         put   90.000000  27.0000  27.0000
159   Default   89.000000         get   89.000000  26.0000  26.0000
420   Default   78.000000         toy   78.000000  25.0000  25.0000
169   Default   76.000000         got   76.000000  24.0000  24.0000
216   Default   68.000000         let   68.000000  23.0000  23.0000
84    Default   67.000000        cool   67.000000  22.0000  22.0000
305   Default   64.000000      pretty   64.000000  21.0000  21.0000
340   Default   61.000000         see   61.000000  20.0000  20.0000
453   Default   60.000000         way   60.000000  19.0000  19.0000
452   Default   60.000000       water   60.000000  18.0000  18.0000
261   Default   57.000000        need   57.000000  17.0000  17.0000
448   Default   52.000000        want   52.000000  16.0000  16.0000
26    Default   52.000000        ball   52.000000  15.0000  15.0000
308   Default   49.000000    probably   49.000000  14.0000  14.0000
403   Default   48.000000        tank   48.000000  13.0000  13.0000
221   Default   79.000000      little   79.000000  12.0000  12.0000
104   Default   49.000000        draw   49.000000  11.0000  11.0000
24    Default   56.000000        back   56.000000  10.0000  10.0000
410   Default   45.000000      though   45.000000   9.0000   9.0000
396   Default   43.000000        sure   43.000000   8.0000   8.0000
225   Default   44.000000       looks   44.000000   7.0000   7.0000
392   Default   41.000000       stuff   41.000000   6.0000   6.0000
438   Default   41.000000         use   41.000000   5.0000   5.0000
252   Default   40.000000      motion   40.000000   4.0000   4.0000
63    Default   41.000000         car   41.000000   3.0000   3.0000
414   Default   39.000000        time   39.000000   2.0000   2.0000
467   Default   37.000000        work   37.000000   1.0000   1.0000
...       ...         ...         ...         ...      ...      ...
403    Topic4   47.483498        tank   48.210594   1.4047  -3.7152
396    Topic4   43.233366        sure   43.950081   1.4035  -3.8089
467    Topic4   36.928719        work   37.653489   1.4005  -3.9665
234    Topic4   35.818205         man   36.536977   1.4000  -3.9971
181    Topic4   35.165460        hard   35.888479   1.3995  -4.0155
460    Topic4   34.697252      wheels   35.418821   1.3993  -4.0289
103    Topic4   32.783536        done   33.521618   1.3976  -4.0856
44     Topic4   31.876921        boat   32.602865   1.3974  -4.1137
167    Topic4   32.365369       going   33.105073   1.3973  -4.0984
383    Topic4   31.505748       start   32.234972   1.3970  -4.1254
82     Topic4   28.380564     control   29.095646   1.3950  -4.2298
349    Topic4   27.853534       shoot   28.580941   1.3941  -4.2486
431    Topic4   27.642207      trying   28.376391   1.3937  -4.2562
204    Topic4   26.946112        kids   27.662163   1.3937  -4.2817
246    Topic4   24.859758       might   25.585405   1.3911  -4.3623
223    Topic4   24.807235        look   25.534517   1.3910  -4.3644
432    Topic4   22.186194        turn   22.911518   1.3877  -4.4761
280    Topic4   21.550277        part   22.275365   1.3868  -4.5051
121    Topic4   21.194959  everything   21.924062   1.3861  -4.5218
235    Topic4   20.374058        many   21.094295   1.3852  -4.5613
275    Topic4   19.335883    

## 2.4 Visualize D-C Brainstorming Topics Using TF-IDF (weighted)

In [23]:
pyLDAvis.sklearn.prepare(lda_tfidf_grp2, dtm_tfidf_grp2, tfidf_vectorizer_grp2)

PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
2      26.125683        1       1  0.333100 -0.062639
3      25.586368        1       2 -0.017864  0.024440
0      25.292376        1       3 -0.207127 -0.256838
1      22.995574        1       4 -0.108109  0.295038, topic_info=     Category       Freq       Term      Total  loglift  logprob
term                                                            
60    Default  77.000000       cool  77.000000  30.0000  30.0000
137   Default  68.000000       kind  68.000000  29.0000  29.0000
130   Default  56.000000       idea  56.000000  28.0000  28.0000
201   Default  58.000000        put  58.000000  27.0000  27.0000
162   Default  59.000000       mean  59.000000  26.0000  26.0000
115   Default  54.000000        got  54.000000  25.0000  25.0000
271   Default  46.000000      water  46.000000  24.0000  24.0000
254   Default  42.000000        toy  42.000000  23.0000  23.0000
263   Default  36.000000        two  36.000000  22.0000  22.0000
43    Default  37.000000        car  37.000000  21.0000  21.0000
270   Default  35.000000       want  35.000000  20.0000  20.0000
70    Default  32.000000       draw  32.000000  19.0000  19.0000
106   Default  29.000000       game  29.000000  18.0000  18.0000
77    Default  28.000000       else  28.000000  17.0000  17.0000
235   Default  30.000000      start  30.000000  16.0000  16.0000
273   Default  30.000000       well  30.000000  15.0000  15.0000
265   Default  26.000000        use  26.000000  14.0000  14.0000
167   Default  29.000000     motion  29.000000  13.0000  13.0000
149   Default  30.000000     little  30.000000  12.0000  12.0000
239   Default  26.000000      stuff  26.000000  11.0000  11.0000
223   Default  26.000000     shoots  26.000000  10.0000  10.0000
113   Default  26.000000       goes  26.000000   9.0000   9.0000
122   Default  26.000000        guy  26.000000   8.0000   8.0000
190   Default  26.000000       play  26.000000   7.0000   7.0000
214   Default  26.000000        see  26.000000   6.0000   6.0000
212   Default  25.000000        say  25.000000   5.0000   5.0000
225   Default  24.000000       side  24.000000   4.0000   4.0000
5     Default  24.000000    alright  24.000000   3.0000   3.0000
221   Default  23.000000      shoot  23.000000   2.0000   2.0000
65    Default  21.000000  different  21.000000   1.0000   1.0000
...       ...        ...        ...        ...      ...      ...
130    Topic4  56.193995       idea  56.903518   1.4573  -2.7731
106    Topic4  28.697088       game  29.413351   1.4452  -3.4451
77     Topic4  27.698230       else  28.400592   1.4448  -3.4806
265    Topic4  25.945148        use  26.653649   1.4429  -3.5459
65     Topic4  20.982232  different  21.689093   1.4367  -3.7582
197    Topic4  19.991966     pretty  20.695395   1.4353  -3.8066
136    Topic4  19.659548       kids  20.356545   1.4350  -3.8234
272    Topic4  19.199632        way  19.904718   1.4338  -3.8470
241    Topic4  18.710896       sure  19.409931   1.4332  -3.8728
89     Topic4  17.892655        fan  18.590964   1.4316  -3.9175
133    Topic4  18.038868    instead  18.744028   1.4315  -3.9094
242    Topic4  18.154207       take  18.867806   1.4313  -3.9030
260    Topic4  16.644723        try  17.355264   1.4281  -3.9898
255    Topic4  16.527071       toys  17.233042   1.4280  -3.9969
284    Topic4  15.618956      write  16.315358   1.4262  -4.0534
160    Topic4  15.423118        man  16.126285   1.4253  -4.0661
114    Topic4  15.173093      going  15.884431   1.4241  -4.0824
153    Topic4  14.986924        lot  15.692784   1.4238  -4.0947
172    Topic4  13.704168       much  14.404623   1.4200  -4.1842
127    Topic4  13.546180        hit  14.269935   1.4178  -4.1958
276    Topic4  12.879103      whole  13.581078   1.4168  -4.2463
2      Topic4  12.849513        add  13.554222   1.4165  -4.2486
116    Topic4  12.447721      gotta  13.145980   1.4153  -

In [24]:
filesListFreeDiv = openFiles(fileNamesFreeDivergent, textPathFree)
filesListFreeConv = openFiles(fileNamesFreeConvergent, textPathFree)
filesListDCDiv = openFiles(fileNamesDCDivergent, textPathDC)
filesListDCConv = openFiles(fileNamesDCConvergent, textPathDC)

group1_raw_div = removeSpecials(filesListFreeDiv)
group1_raw_conv = removeSpecials(filesListFreeConv)
group2_raw_div = removeSpecials(filesListDCDiv)
group2_raw_conv = removeSpecials(filesListDCConv)

group1Div = removeObservations(group1_raw_div)
group1Conv = removeObservations(group1_raw_conv)
group2Div = removeObservations(group2_raw_div)
group2Conv = removeObservations(group2_raw_conv)

group1_div_lines = group1Div.split('\n')
group1_conv_lines = group1Conv.split('\n')
group2_div_lines = group2Div.split('\n')
group2_conv_lines = group2Conv.split('\n')

print("Number of lines in the Free Group (divergent process):", len(group1_div_lines))
print("Number of lines in the Free Group (convergent process):", len(group1_conv_lines))
print("Number of lines in the D-C Group (divergent process):", len(group2_div_lines))
print("Number of lines in the D-C Group (convergent process):", len(group2_conv_lines))
print("Each line will be considered a separate document for computing the Document-Term Matrix")

Number of lines in the Free Group (divergent process): 5023
Number of lines in the Free Group (convergent process): 2619
Number of lines in the D-C Group (divergent process): 3872
Number of lines in the D-C Group (convergent process): 2769
Each line will be considered a separate document for computing the Document-Term Matrix


In [25]:
tf_vectorizer_grp1_div = CountVectorizer(strip_accents = 'unicode',
                                         analyzer = 'word',
                                         stop_words = complete_stop_words,
                                         lowercase = True,
                                         token_pattern = r'\b[a-zA-Z]{3,}\b',
                                         max_df = 0.5, 
                                         min_df = 10)

tf_vectorizer_grp1_conv = CountVectorizer(strip_accents = 'unicode',
                                          analyzer = 'word',
                                          stop_words = complete_stop_words,
                                          lowercase = True,
                                          token_pattern = r'\b[a-zA-Z]{3,}\b',
                                          max_df = 0.5, 
                                          min_df = 10)

tf_vectorizer_grp2_div = CountVectorizer(strip_accents = 'unicode',
                                         analyzer = 'word',
                                         stop_words = complete_stop_words,
                                         lowercase = True,
                                         token_pattern = r'\b[a-zA-Z]{3,}\b',
                                         max_df = 0.5, 
                                         min_df = 10)

tf_vectorizer_grp2_conv = CountVectorizer(strip_accents = 'unicode',
                                          analyzer = 'word',
                                          stop_words = complete_stop_words,
                                          lowercase = True,
                                          token_pattern = r'\b[a-zA-Z]{3,}\b',
                                          max_df = 0.5, 
                                          min_df = 10)
dtm_tf_grp1_div = tf_vectorizer_grp1.fit_transform(group1_div_lines)
print("Document-Term Matrix size for Free Group (divergent process):", dtm_tf_grp1_div.shape)
dtm_tf_grp1_conv = tf_vectorizer_grp1.fit_transform(group1_conv_lines)
print("Document-Term Matrix size for Free Group (convergent process):", dtm_tf_grp1_conv.shape)

dtm_tf_grp2_div = tf_vectorizer_grp2.fit_transform(group2_div_lines)
print("Document-Term Matrix size for Free Group (divergent process):", dtm_tf_grp2_div.shape)
dtm_tf_grp2_conv = tf_vectorizer_grp2.fit_transform(group2_conv_lines)
print("Document-Term Matrix size for Free Group (convergent process):", dtm_tf_grp2_conv.shape)

Document-Term Matrix size for Free Group (divergent process): (5023, 353)
Document-Term Matrix size for Free Group (convergent process): (2619, 159)
Document-Term Matrix size for Free Group (divergent process): (3872, 285)
Document-Term Matrix size for Free Group (convergent process): (2769, 184)


In [26]:
tfidf_vectorizer_grp1_div = TfidfVectorizer(**tf_vectorizer_grp1_div.get_params())
dtm_tfidf_grp1_div = tfidf_vectorizer_grp1_div.fit_transform(group1_div_lines)
tfidf_vectorizer_grp1_conv = TfidfVectorizer(**tf_vectorizer_grp1_conv.get_params())
dtm_tfidf_grp1_conv = tfidf_vectorizer_grp1_conv.fit_transform(group1_conv_lines)

tfidf_vectorizer_grp2_div = TfidfVectorizer(**tf_vectorizer_grp2_div.get_params())
dtm_tfidf_grp2_div = tfidf_vectorizer_grp2_div.fit_transform(group2_div_lines)
tfidf_vectorizer_grp2_conv = TfidfVectorizer(**tf_vectorizer_grp2_conv.get_params())
dtm_tfidf_grp2_conv = tfidf_vectorizer_grp2_conv.fit_transform(group2_conv_lines)

# for TFIDF DTM
lda_tfidf_grp1_div = LatentDirichletAllocation(n_topics=4, random_state=0)
lda_tfidf_grp1_div.fit(dtm_tfidf_grp1_div)
lda_tfidf_grp1_conv = LatentDirichletAllocation(n_topics=4, random_state=0)
lda_tfidf_grp1_conv.fit(dtm_tfidf_grp1_conv)

lda_tfidf_grp2_div = LatentDirichletAllocation(n_topics=4, random_state=0)
lda_tfidf_grp2_div.fit(dtm_tfidf_grp2_div)
lda_tfidf_grp2_conv = LatentDirichletAllocation(n_topics=4, random_state=0)
lda_tfidf_grp2_conv.fit(dtm_tfidf_grp2_conv)

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
             evaluate_every=-1, learning_decay=0.7,
             learning_method='online', learning_offset=10.0,
             max_doc_update_iter=100, max_iter=10, mean_change_tol=0.001,
             n_jobs=1, n_topics=4, perp_tol=0.1, random_state=0,
             topic_word_prior=None, total_samples=1000000.0, verbose=0)

In [27]:
pyLDAvis.sklearn.prepare(lda_tfidf_grp1_div, dtm_tfidf_grp1_div, tfidf_vectorizer_grp1_div)

PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
2      26.114140        1       1 -0.091936  0.084398
0      25.763366        1       2 -0.116285  0.246398
1      25.762000        1       3 -0.149647 -0.305002
3      22.360493        1       4  0.357868 -0.025794, topic_info=     Category       Freq       Term      Total  loglift  logprob
term                                                            
144   Default  80.000000       idea  80.000000  30.0000  30.0000
239   Default  69.000000        put  69.000000  29.0000  29.0000
337   Default  64.000000      water  64.000000  28.0000  28.0000
184   Default  70.000000       mean  70.000000  27.0000  27.0000
312   Default  62.000000        toy  62.000000  26.0000  26.0000
166   Default  60.000000     little  60.000000  25.0000  25.0000
340   Default  59.000000       well  59.000000  24.0000  24.0000
63    Default  50.000000       cool  50.000000  23.0000  23.0000
124   Default  50.000000        got  50.000000  22.0000  22.0000
48    Default  42.000000        car  42.000000  21.0000  21.0000
232   Default  45.000000     pretty  45.000000  20.0000  20.0000
338   Default  42.000000        way  42.000000  19.0000  19.0000
153   Default  41.000000       kind  41.000000  18.0000  18.0000
117   Default  55.000000        get  55.000000  17.0000  17.0000
19    Default  40.000000       ball  40.000000  16.0000  16.0000
162   Default  49.000000        let  49.000000  15.0000  15.0000
46    Default  32.000000      candy  32.000000  14.0000  14.0000
311   Default  31.000000        top  31.000000  13.0000  13.0000
192   Default  34.000000     motion  34.000000  12.0000  12.0000
313   Default  35.000000       toys  35.000000  11.0000  11.0000
61    Default  30.000000    control  30.000000  10.0000  10.0000
70    Default  32.000000  different  32.000000   9.0000   9.0000
295   Default  32.000000      stuff  32.000000   8.0000   8.0000
328   Default  28.000000        use  28.000000   7.0000   7.0000
78    Default  31.000000       else  31.000000   6.0000   6.0000
277   Default  30.000000       sort  30.000000   5.0000   5.0000
128   Default  30.000000        gun  30.000000   4.0000   4.0000
200   Default  30.000000       need  30.000000   3.0000   3.0000
265   Default  29.000000     shoots  29.000000   2.0000   2.0000
291   Default  26.000000      still  26.000000   1.0000   1.0000
...       ...        ...        ...        ...      ...      ...
337    Topic4  64.259730      water  64.958190   1.4871  -2.8797
48     Topic4  42.057129        car  42.752048   1.4815  -3.3036
46     Topic4  31.828169      candy  32.520423   1.4764  -3.5823
311    Topic4  30.570332        top  31.270214   1.4752  -3.6226
61     Topic4  29.706671    control  30.405649   1.4746  -3.6513
328    Topic4  27.926614        use  28.627132   1.4731  -3.7131
291    Topic4  25.343006      still  26.035388   1.4709  -3.8102
146    Topic4  22.315496     inside  23.010460   1.4672  -3.9374
72     Topic4  22.259297       done  22.960557   1.4669  -3.9399
152    Topic4  21.820408       kids  22.519855   1.4663  -3.9598
322    Topic4  21.921826     trying  22.625409   1.4663  -3.9552
170    Topic4  19.756994        lot  20.462854   1.4628  -4.0592
2      Topic4  18.338452        add  19.038487   1.4604  -4.1337
305    Topic4  16.226512    thought  16.920396   1.4560  -4.2560
306    Topic4  15.424050      three  16.112534   1.4542  -4.3067
272    Topic4  15.234860      small  15.932837   1.4531  -4.3191
245    Topic4  14.502086     rocket  15.191816   1.4514  -4.3684
203    Topic4  14.195814        new  14.894695   1.4498  -4.3897
112    Topic4  13.833142       four  14.522368   1.4493  -4.4156
315    Topic4  13.097297      train  13.784403   1.4467  -4.4703
247    Topic4  13.068788       roll  13.776242   1.4452  -4.4724
142    Topic4  12.639601       hold  13.327544   1.4449  -4.5058
20     Topic4  12.569607    balloon  13.257726   1.4446  -

In [28]:
pyLDAvis.sklearn.prepare(lda_tfidf_grp1_conv, dtm_tfidf_grp1_conv, tfidf_vectorizer_grp1_conv)

PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
0      25.939579        1       1 -0.139376 -0.214382
2      24.960957        1       2 -0.128695  0.323674
1      24.814942        1       3  0.360841  0.003608
3      24.284523        1       4 -0.092770 -0.112900, topic_info=     Category       Freq        Term      Total  loglift  logprob
term                                                             
149   Default  47.000000        well  47.000000  30.0000  30.0000
51    Default  46.000000         get  46.000000  29.0000  29.0000
79    Default  44.000000        mean  44.000000  28.0000  28.0000
62    Default  39.000000        idea  39.000000  27.0000  27.0000
142   Default  38.000000         two  38.000000  26.0000  26.0000
127   Default  37.000000        tank  37.000000  25.0000  25.0000
54    Default  33.000000         got  33.000000  24.0000  24.0000
87    Default  28.000000        need  28.000000  23.0000  23.0000
100   Default  29.000000    probably  29.000000  22.0000  22.0000
45    Default  26.000000       final  26.000000  21.0000  21.0000
35    Default  26.000000        draw  26.000000  20.0000  20.0000
99    Default  26.000000      pretty  26.000000  19.0000  19.0000
102   Default  31.000000         put  31.000000  18.0000  18.0000
71    Default  26.000000      little  26.000000  17.0000  17.0000
136   Default  25.000000         toy  25.000000  16.0000  16.0000
146   Default  25.000000        want  25.000000  15.0000  15.0000
107   Default  24.000000         see  24.000000  14.0000  14.0000
11    Default  22.000000        back  22.000000  13.0000  13.0000
74    Default  22.000000       looks  22.000000  12.0000  12.0000
153   Default  22.000000        wind  22.000000  11.0000  11.0000
148   Default  21.000000         way  21.000000  10.0000  10.0000
30    Default  21.000000        cool  21.000000   9.0000   9.0000
77    Default  21.000000         man  21.000000   8.0000   8.0000
69    Default  21.000000         let  21.000000   7.0000   7.0000
130   Default  20.000000      though  20.000000   6.0000   6.0000
29    Default  19.000000     concept  19.000000   5.0000   5.0000
119   Default  19.000000      spring  19.000000   4.0000   4.0000
25    Default  17.000000        come  17.000000   3.0000   3.0000
12    Default  17.000000         bad  17.000000   2.0000   2.0000
156   Default  17.000000        work  17.000000   1.0000   1.0000
...       ...        ...         ...        ...      ...      ...
45     Topic4  25.853673       final  26.550318   1.3887  -3.0143
11     Topic4  22.219741        back  22.929033   1.3839  -3.1657
74     Topic4  22.200145       looks  22.917023   1.3835  -3.1666
29     Topic4  18.801601     concept  19.503829   1.3787  -3.3328
25     Topic4  16.573206        come  17.286520   1.3732  -3.4589
126    Topic4  14.736860        take  15.437897   1.3689  -3.5764
42     Topic4  14.658411        even  15.371939   1.3678  -3.5817
63     Topic4  14.490413       ideas  15.197169   1.3677  -3.5932
36     Topic4  14.155038     drawing  14.854361   1.3671  -3.6166
6      Topic4  14.086087    anything  14.787454   1.3667  -3.6215
158    Topic4  13.490306       write  14.189573   1.3648  -3.6647
138    Topic4  12.913345        true  13.608802   1.3629  -3.7084
131    Topic4  12.595339     thought  13.294591   1.3613  -3.7334
44     Topic4  10.654074        feel  11.353147   1.3518  -3.9008
17     Topic4  10.661984         big  11.363443   1.3516  -3.9000
43     Topic4  10.588120  everything  11.297756   1.3505  -3.9070
19     Topic4  10.193150        boat  10.907198   1.3476  -3.9450
82     Topic4   9.241656     minutes   9.941369   1.3423  -4.0430
15     Topic4   8.911073        best   9.613769   1.3394  -4.0794
46     Topic4   8.858376       first   9.561208   1.3390  -4.0853
33     Topic4   8.181985   difficult   8.892280   1.3321  -4.1648
40     Topic4   7.944218        else   8.642367   1.3311  -4.1943
4  

In [29]:
pyLDAvis.sklearn.prepare(lda_tfidf_grp2_div, dtm_tfidf_grp2_div, tfidf_vectorizer_grp2_div)

PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
2      26.125683        1       1  0.333100 -0.062639
3      25.586368        1       2 -0.017864  0.024440
0      25.292376        1       3 -0.207127 -0.256838
1      22.995574        1       4 -0.108109  0.295038, topic_info=     Category       Freq       Term      Total  loglift  logprob
term                                                            
60    Default  77.000000       cool  77.000000  30.0000  30.0000
137   Default  68.000000       kind  68.000000  29.0000  29.0000
130   Default  56.000000       idea  56.000000  28.0000  28.0000
201   Default  58.000000        put  58.000000  27.0000  27.0000
162   Default  59.000000       mean  59.000000  26.0000  26.0000
115   Default  54.000000        got  54.000000  25.0000  25.0000
271   Default  46.000000      water  46.000000  24.0000  24.0000
254   Default  42.000000        toy  42.000000  23.0000  23.0000
263   Default  36.000000        two  36.000000  22.0000  22.0000
43    Default  37.000000        car  37.000000  21.0000  21.0000
270   Default  35.000000       want  35.000000  20.0000  20.0000
70    Default  32.000000       draw  32.000000  19.0000  19.0000
106   Default  29.000000       game  29.000000  18.0000  18.0000
77    Default  28.000000       else  28.000000  17.0000  17.0000
235   Default  30.000000      start  30.000000  16.0000  16.0000
273   Default  30.000000       well  30.000000  15.0000  15.0000
265   Default  26.000000        use  26.000000  14.0000  14.0000
167   Default  29.000000     motion  29.000000  13.0000  13.0000
149   Default  30.000000     little  30.000000  12.0000  12.0000
239   Default  26.000000      stuff  26.000000  11.0000  11.0000
223   Default  26.000000     shoots  26.000000  10.0000  10.0000
113   Default  26.000000       goes  26.000000   9.0000   9.0000
122   Default  26.000000        guy  26.000000   8.0000   8.0000
190   Default  26.000000       play  26.000000   7.0000   7.0000
214   Default  26.000000        see  26.000000   6.0000   6.0000
212   Default  25.000000        say  25.000000   5.0000   5.0000
225   Default  24.000000       side  24.000000   4.0000   4.0000
5     Default  24.000000    alright  24.000000   3.0000   3.0000
221   Default  23.000000      shoot  23.000000   2.0000   2.0000
65    Default  21.000000  different  21.000000   1.0000   1.0000
...       ...        ...        ...        ...      ...      ...
130    Topic4  56.193995       idea  56.903518   1.4573  -2.7731
106    Topic4  28.697088       game  29.413351   1.4452  -3.4451
77     Topic4  27.698230       else  28.400592   1.4448  -3.4806
265    Topic4  25.945148        use  26.653649   1.4429  -3.5459
65     Topic4  20.982232  different  21.689093   1.4367  -3.7582
197    Topic4  19.991966     pretty  20.695395   1.4353  -3.8066
136    Topic4  19.659548       kids  20.356545   1.4350  -3.8234
272    Topic4  19.199632        way  19.904718   1.4338  -3.8470
241    Topic4  18.710896       sure  19.409931   1.4332  -3.8728
89     Topic4  17.892655        fan  18.590964   1.4316  -3.9175
133    Topic4  18.038868    instead  18.744028   1.4315  -3.9094
242    Topic4  18.154207       take  18.867806   1.4313  -3.9030
260    Topic4  16.644723        try  17.355264   1.4281  -3.9898
255    Topic4  16.527071       toys  17.233042   1.4280  -3.9969
284    Topic4  15.618956      write  16.315358   1.4262  -4.0534
160    Topic4  15.423118        man  16.126285   1.4253  -4.0661
114    Topic4  15.173093      going  15.884431   1.4241  -4.0824
153    Topic4  14.986924        lot  15.692784   1.4238  -4.0947
172    Topic4  13.704168       much  14.404623   1.4200  -4.1842
127    Topic4  13.546180        hit  14.269935   1.4178  -4.1958
276    Topic4  12.879103      whole  13.581078   1.4168  -4.2463
2      Topic4  12.849513        add  13.554222   1.4165  -4.2486
116    Topic4  12.447721      gotta  13.145980   1.4153  -

In [30]:
pyLDAvis.sklearn.prepare(lda_tfidf_grp2_conv, dtm_tfidf_grp2_conv, tfidf_vectorizer_grp2_conv)

PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
0      26.666333        1       1 -0.144470 -0.093549
2      25.976853        1       2 -0.227331 -0.089320
1      24.903129        1       3  0.318795 -0.163714
3      22.453685        1       4  0.053006  0.346582, topic_info=     Category       Freq        Term      Total  loglift  logprob
term                                                             
130   Default  49.000000         put  49.000000  30.0000  30.0000
26    Default  45.000000         car  45.000000  29.0000  29.0000
90    Default  40.000000        kind  40.000000  28.0000  28.0000
73    Default  39.000000         got  39.000000  27.0000  27.0000
172   Default  41.000000         two  41.000000  26.0000  26.0000
92    Default  35.000000         let  35.000000  25.0000  25.0000
105   Default  36.000000        mean  36.000000  24.0000  24.0000
81    Default  35.000000        idea  35.000000  23.0000  23.0000
35    Default  34.000000        cool  34.000000  22.0000  22.0000
112   Default  37.000000        need  37.000000  21.0000  21.0000
176   Default  36.000000        want  36.000000  20.0000  20.0000
57    Default  32.000000       final  32.000000  19.0000  19.0000
68    Default  36.000000         get  36.000000  18.0000  18.0000
139   Default  26.000000         see  26.000000  17.0000  17.0000
38    Default  27.000000   different  27.000000  16.0000  16.0000
179   Default  27.000000        well  27.000000  15.0000  15.0000
32    Default  25.000000     concept  25.000000  14.0000  14.0000
178   Default  25.000000         way  25.000000  13.0000  13.0000
40    Default  25.000000        draw  25.000000  12.0000  12.0000
159   Default  25.000000        take  25.000000  11.0000  11.0000
160   Default  23.000000    thinking  23.000000  10.0000  10.0000
122   Default  23.000000       point  23.000000   9.0000   9.0000
44    Default  21.000000    elephant  21.000000   8.0000   8.0000
94    Default  20.000000      little  20.000000   7.0000   7.0000
181   Default  22.000000      wheels  22.000000   6.0000   6.0000
13    Default  21.000000        back  21.000000   5.0000   5.0000
82    Default  20.000000       ideas  20.000000   4.0000   4.0000
158   Default  20.000000        sure  20.000000   3.0000   3.0000
137   Default  20.000000         say  20.000000   2.0000   2.0000
153   Default  19.000000       start  19.000000   1.0000   1.0000
...       ...        ...         ...        ...      ...      ...
139    Topic4  25.982770         see  26.683218   1.4671  -3.1154
121    Topic4  16.005783        play  16.711063   1.4506  -3.5999
173    Topic4  16.089743         use  16.800025   1.4505  -3.5947
12     Topic4  15.453117     awesome  16.148753   1.4497  -3.6351
67     Topic4  14.946553        game  15.650761   1.4477  -3.6684
177    Topic4  14.798988       water  15.504414   1.4471  -3.6783
50     Topic4  12.918998  everything  13.619818   1.4409  -3.8142
123    Topic4  13.189456      pretty  13.907369   1.4407  -3.7934
157    Topic4  12.740090    supposed  13.440163   1.4402  -3.8281
4      Topic4  12.530183     already  13.223787   1.4398  -3.8447
117    Topic4  12.531136      peanut  13.224887   1.4398  -3.8447
29     Topic4  12.620093     combine  13.325686   1.4393  -3.8376
108    Topic4  11.591915      motion  12.292572   1.4350  -3.9226
165    Topic4  11.597600         top  12.308245   1.4342  -3.9221
98     Topic4  11.382480       looks  12.082692   1.4340  -3.9408
53     Topic4  11.233742         fan  11.938856   1.4328  -3.9539
19     Topic4  16.859157        blue  17.937786   1.4317  -3.5480
46     Topic4  10.526105         end  11.237143   1.4283  -4.0190
74     Topic4  10.578396       gotta  11.295734   1.4281  -4.0141
34     Topic4   9.959225     control  10.656761   1.4260  -4.0744
136    Topic4   9.676902        said  10.375534   1.4240  -4.1031
102    Topic4   9.584367         man  10.278063   1.4238  -4.1127
1  